# Imports

In [3]:
import pandas as pd
import h5py
import numpy as np
import tensorflow as tf
import h5py
import os
from sklearn.metrics import accuracy_score
import ecg_plot

In [8]:
#Lanzar desde la carpeta LightX3ECGPrivate
import sys
sys.path.append('./automatic-ecg-diagnosis/')
# sys.path.append('./source/')
import predict

AttributeError: module 'tensorflow.compat.v2' has no attribute '__internal__'

# Commands

In [2]:
!git clone https://github.com/antonior92/automatic-ecg-diagnosis.git

Cloning into 'automatic-ecg-diagnosis'...


In [7]:
#Comando windows
!py -m pip install tensorflow

^C


Defaulting to user installation because normal site-packages is not writeable


In [6]:
!cd

e:\automatic-ecg-diagnosis-private


In [25]:
%cd automatic-ecg-diagnosis

e:\automatic-ecg-diagnosis-private\automatic-ecg-diagnosis


In [2]:
!ls

"ls" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [11]:
!wget https://www.dropbox.com/s/p3vd3plcbu9sf1o/data.zip?dl=0 -O data.zip

--2023-05-02 18:21:58--  https://www.dropbox.com/s/p3vd3plcbu9sf1o/data.zip?dl=0
Resolviendo www.dropbox.com (www.dropbox.com)... 162.125.68.18, 2620:100:6024:18::a27d:4412
Conectando con www.dropbox.com (www.dropbox.com)[162.125.68.18]:443... conectado.
Petición HTTP enviada, esperando respuesta... 302 Found
Localización: /s/raw/p3vd3plcbu9sf1o/data.zip [siguiendo]
--2023-05-02 18:21:58--  https://www.dropbox.com/s/raw/p3vd3plcbu9sf1o/data.zip
Reutilizando la conexión con www.dropbox.com:443.
Petición HTTP enviada, esperando respuesta... 302 Found
Localización: https://uc776bd503b4554baf42e8e1444b.dl.dropboxusercontent.com/cd/0/inline/B7Qub56Dm4nFxk8XRoEMlBkzsCI3PaHncyWZKodICaR55uM9eMuYuoUZnH-9USPaOcJJ6qJM3mylVU6u86B3BR2k6jWUil7g29fotfKh0d6Kir86bKWE2Qd4ddmw2mvBrzpm5X-3y240SOomVSv8E0sCPCoBATC3cVbe2ZJF0MvPxg/file# [siguiendo]
--2023-05-02 18:21:59--  https://uc776bd503b4554baf42e8e1444b.dl.dropboxusercontent.com/cd/0/inline/B7Qub56Dm4nFxk8XRoEMlBkzsCI3PaHncyWZKodICaR55uM9eMuYuoUZnH-9USP

In [12]:
!unzip data.zip

Archive:  data.zip
  inflating: data/ecg_tracings.hdf5  
replace data/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [8]:
!pip3 show tensorflow

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/tfg_luis_2223/.local/lib/python3.7/site-packages
Requires: google-pasta, astor, gast, keras-preprocessing, six, protobuf, wheel, keras-applications, absl-py, tensorboard, wrapt, grpcio, numpy, termcolor, tensorflow-estimator
Required-by: 


In [5]:
!python install -U pip

python: can't open file 'install': [Errno 2] No such file or directory


In [7]:
!python -v

# installing zipimport hook
import zipimport # builtin
# installed zipimport hook
# /usr/lib/python2.7/site.pyc matches /usr/lib/python2.7/site.py
import site # precompiled from /usr/lib/python2.7/site.pyc
# /usr/lib/python2.7/os.pyc matches /usr/lib/python2.7/os.py
import os # precompiled from /usr/lib/python2.7/os.pyc
import errno # builtin
import posix # builtin
# /usr/lib/python2.7/posixpath.pyc matches /usr/lib/python2.7/posixpath.py
import posixpath # precompiled from /usr/lib/python2.7/posixpath.pyc
# /usr/lib/python2.7/stat.pyc matches /usr/lib/python2.7/stat.py
import stat # precompiled from /usr/lib/python2.7/stat.pyc
# /usr/lib/python2.7/genericpath.pyc matches /usr/lib/python2.7/genericpath.py
import genericpath # precompiled from /usr/lib/python2.7/genericpath.pyc
# /usr/lib/python2.7/warnings.pyc matches /usr/lib/python2.7/warnings.py
import warnings # precompiled from /usr/lib/python2.7/warnings.pyc
# /usr/lib/python2.7/linecache.pyc matches /usr/lib/python2.7/linecache.

# Code

## crear hdf5

In [48]:
filename = 'data/data2.hdf5'
f = h5py.File(filename, 'w')

In [49]:
folder_path = 'CasosNumpy'
train = pd.read_csv("data/annotations/train.csv")
val = pd.read_csv("data/annotations/val.csv")
df = pd.concat([train,val],ignore_index=True,)
group_name = 'tracings'
shape = (len(df['id'].values),4096,12)
dset = f.create_dataset(group_name,shape=shape)

count = 0
for file_name in os.listdir(folder_path):
    if(file_name[:5] in df['id'].values):
        if file_name.endswith('.npy'):
            # Cargar el archivo numpy
            arr = np.load(os.path.join(folder_path, file_name))[:,:4096]
            if arr.shape != (12, 4096):
                new_arr = np.zeros((12, 4096))
                new_arr[:arr.shape[0], :arr.shape[1]] = arr
                arr = new_arr
            #arr = tf.convert_to_tensor(arr, dtype=tf.float32)
            dset[count, :, :] = arr.transpose()
            count+=1
            
# Guardar el arreglo numpy en el grupo

In [50]:
print(len(f[group_name]))
f.close()

6877


## Modificar val.csv a como lo tiene antonio

In [44]:
val = pd.read_csv("data/annotations/val.csv")
#val = val.drop(['id','age','sex','r_count','length'],axis=1)

train = pd.read_csv("data/annotations/train.csv")
#print(val)
#print(train)
final = pd.concat([train,val],ignore_index=True)
final = final.sort_values('id',)
final = final.reset_index()
final = final.drop(['index','id','age','sex','r_count','length'],axis=1)

print(final)

      label_0  label_1  label_2  label_3  label_4  label_5  label_6  label_7   
0           0        0        0        0        1        0        0        0  \
1           1        0        0        0        0        0        0        0   
2           0        1        0        0        0        0        0        0   
3           0        1        0        0        0        0        0        0   
4           0        0        0        0        0        0        1        0   
...       ...      ...      ...      ...      ...      ...      ...      ...   
6872        0        0        1        0        0        0        0        0   
6873        0        0        0        0        0        0        0        1   
6874        0        0        0        1        0        0        0        0   
6875        0        1        0        0        0        0        0        0   
6876        0        0        0        0        0        0        1        0   

      label_8  
0           0  
1      

In [45]:
final.to_csv("data/annotations/train_CPSC.csv",)


## Otros

In [2]:
args= "datasets/pred.hdf5"
with h5py.File(args, "r") as f:
    x = np.array(f['tracings'])

np.size(x[1])
x.shape

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'datasets/pred.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [26]:
npy = np.load("./history.npy")
npy

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [15]:
def ecgPlot(data,sample, factor = 1):
    data = data* factor
    print(data.shape)
    #print(data)
    xml_leads = ['DI', 'DII', 'DIII', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    ecg_plot.plot(data.T, sample_rate= sample,lead_index=xml_leads, title=f"{filename}")
    ecg_plot.save_as_png("ecg")

In [18]:
filename = './automatic-ecg-diagnosis/data/train.hdf5'
f = h5py.File(filename, 'r')

In [23]:
ecgPlot(f['tracings'][1],400)

(4096, 12)


# Train

In [21]:
!py train.py ./data/data.hdf5 ./data/annotations/train_CPSC.csv --val_split 0.2


54/54 [==============================] - 379s 7s/step - loss: -12790.8604 - val_loss: -69358.4297 - lr: 0.0010

86/86 [==============================] - 505s 6s/step - loss: -34734.5156 - val_loss: -184735.9688 - lr: 0.0010


In [31]:
!py train.py -h

usage: train.py [-h] [--val_split VAL_SPLIT] [--dataset_name DATASET_NAME]
                path_to_hdf5 path_to_csv

Train neural network.

positional arguments:
  path_to_hdf5          path to hdf5 file containing tracings
  path_to_csv           path to csv file containing annotations

optional arguments:
  -h, --help            show this help message and exit
  --val_split VAL_SPLIT
                        number between 0 and 1 determining how much of the
                        data is to be used for validation. The remaining is
                        used for validation. Default: 0.02
  --dataset_name DATASET_NAME
                        name of the hdf5 dataset containing tracings


# Predict

In [2]:
PFOLDER="./dnn_predicts"
DFOLDER="./data" #Nos hemos cambiado al directorio de automatic-ecg-diagnosis

!python predict.py $DFOLDER/data.hdf5 final_model_.hdf5 --output_file $PFOLDER/train_val.npy


215/215 [==============================] - 75s 344ms/step
Output predictions saved


2023-05-15 10:31:22.661074: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
PFOLDER="./dnn_predicts"
DFOLDER="./data" #Nos hemos cambiado al directorio de automatic-ecg-diagnosis

!python predict.py $DFOLDER/ecg_tracings2.hdf5 model.hdf5 --output_file $PFOLDER/ecg_tracings.npy

In [10]:
PFOLDER="./dnn_predicts"
DFOLDER="./data" #Nos hemos cambiado al directorio de automatic-ecg-diagnosis

!python predict.py $DFOLDER/pred.hdf5 final_model_.hdf5 --output_file $PFOLDER/pred.npy


1/1 [==============================] - 1s 621ms/step
Output predictions saved


2023-05-15 10:40:05.019104: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:
!python predict.py -h

usage: predict.py [-h] [--dataset_name DATASET_NAME]
                  [--output_file OUTPUT_FILE] [-bs BS]
                  path_to_hdf5 path_to_model

Get performance on test set from hdf5

positional arguments:
  path_to_hdf5          path to hdf5 file containing tracings
  path_to_model         file containing training model.

optional arguments:
  -h, --help            show this help message and exit
  --dataset_name DATASET_NAME
                        name of the hdf5 dataset containing tracings
  --output_file OUTPUT_FILE
                        output csv file.
  -bs BS                Batch size.


In [17]:
pred = np.load('dnn_predicts/ecg_tracings.npy')
csv = pd.read_csv('data/annotations/dnn.csv')
pos = 22
print(pred[pos,1:])
print(csv.values[pos])


[2.5101798e-03 1.6017100e-05 6.6078037e-01 5.1851600e-04 2.2911260e-07]
[22  0  0  0  1  0  0]


In [23]:
pred = np.load('dnn_predicts/model_pred.npy')
pred[:,0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

In [9]:
pred = np.load('dnn_predicts/train_val.npy')
csv = pd.read_csv('data/annotations/train_CPSC.csv')
pos = 3
print(pred[pos,1:])
print(csv.values[pos,1:])


[9.1813236e-01 2.8199967e-02 1.2204376e-01 5.4315905e-19 4.4130284e-06
 2.0833155e-05 6.2848147e-07 7.4192864e-04 2.2872496e-06]
[0 0 0 0 1 0 0 0 0]


In [17]:
pred = np.load('dnn_predicts/pred.npy')
csv = pd.read_csv('data/annotations/pred.csv')
pos = 10
print(pred[pos,1:])
print(csv.values[pos,1:])

[9.20027375e-01 2.83536166e-02 1.21105045e-01 5.42684766e-19
 4.58824525e-06 2.08452784e-05 6.21495985e-07 7.38178031e-04
 2.34572985e-06]
[65 0 0 0 0 1 0 0 0 0 0 12 5000]


In [22]:
pred = np.load('dnn_predicts/train_val.npy')
csv = pd.read_csv('data/annotations/train_CPSC.csv',)
threshold = pd.read_csv('dnn_predicts/optimal_thresholds_best.csv')
result = pd.DataFrame(data= [])

for pos in range(len(pred)):
    data = pd.DataFrame([pred[pos,1:] >= threshold['Threshold']])
    result = pd.concat([result,data])

result = result.replace({True:1,False:0})
print(result)

              0      1      2      3      4      5      6      7      8
Threshold  True  False  False  False  False  False  False  False  False
Threshold  True  False  False  False  False  False  False  False  False
Threshold  True  False  False  False  False  False  False  False  False
Threshold  True  False  False  False  False  False  False  False  False
Threshold  True  False  False  False  False  False  False  False  False
...         ...    ...    ...    ...    ...    ...    ...    ...    ...
Threshold  True  False  False  False  False  False  False  False  False
Threshold  True  False  False  False  False  False  False  False  False
Threshold  True  False  False  False  False  False  False  False  False
Threshold  True  False  False  False  False  False  False  False  False
Threshold  True  False  False  False  False  False  False  False  False

[6877 rows x 9 columns]
           0  1  2  3  4  5  6  7  8
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0


In [37]:
pred = np.load('dnn_predicts/pred.npy')
csv = pd.read_csv('data/annotations/pred.csv',)
csv = csv.drop(columns=['age','sex','r_count','length'],axis=1)

threshold = pd.read_csv('dnn_predicts/optimal_thresholds_best.csv')
result = pd.DataFrame(data= [])

for pos in range(len(pred)):
    data = pd.DataFrame([pred[pos,1:] >= threshold['Threshold']])
    result = pd.concat([result,data])

result = result.replace({True:1,False:0})
print(result)

           0  1  2  3  4  5  6  7  8
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
Threshold  1  0  0  0  0  0  0  0  0
T

In [27]:
csv.values[:,1:]

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [38]:
accuracy = accuracy_score(csv.values[:,1:],result.values)
accuracy

ValueError: Classification metrics can't handle a mix of unknown and multilabel-indicator targets